<img src="data/images/lecture-notebook-header.png" />

# Clustering: AGNES

The AGNES (Agglomerative Nesting) clustering algorithm is an agglomerative hierarchical clustering algorithm used to group similar objects or data points into clusters. The algorithm follows a bottom-up approach, starting with each data point as a separate cluster and then iteratively merging the closest clusters until a termination condition is met. Here's a step-by-step overview of the AGNES clustering algorithm:

* Begin by considering each data point as a separate cluster.
* Compute the proximity matrix, which represents the distance or dissimilarity between each pair of clusters.
* Find the two closest clusters based on the proximity matrix. This can be done using various distance metrics such as Euclidean distance or Manhattan distance.
* Merge the two closest clusters into a single cluster.
* Update the proximity matrix by recalculating the distances between the merged cluster and the remaining clusters. This step is crucial as it affects the subsequent cluster merging decisions.
* Repeat steps 3-5 until a termination condition is met. This condition could be a predefined number of clusters or a distance threshold that determines the maximum dissimilarity allowed for cluster merging.
* The algorithm terminates when all data points belong to a single cluster or the termination condition is satisfied.

AGNES produces a hierarchical clustering structure in the form of a dendrogram, which represents the merging order of clusters. The dendrogram can be cut at a desired level to obtain a specific number of clusters.

One limitation of AGNES is its computational complexity, especially for large datasets, as the algorithm requires calculating the proximity matrix at each iteration. Another consideration is the choice of distance metric, as it can significantly impact the clustering results.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Import Required Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.datasets import make_blobs, make_moons, make_circles
from scipy.cluster.hierarchy import dendrogram

from tqdm import tqdm

## Playing with Toy Data

We use the very small and manually crafted dataset shown in the lecture slides to keep it simple. Otherwise, the corresponding distance matrix and dendrogram get quickly uncomfortably large to easily comprehend. 

In [ ]:
# Create the datset with the 6 data points
X_demo = np.array([ (-6,1), (-4.8,2.9), (-2,-2.5), (3,2), (2.7,-3.5), (3,-2) ])

# Label the date points from 0..5
labels_demo = list(range(X_demo.shape[0]))

# Plot all data points with their labels
plt.figure()
plt.scatter(X_demo[:,0], X_demo[:,1])
for i, label in enumerate(labels_demo):
    plt.gca().annotate(label, X_demo[i], fontsize=14)
plt.show()

Let's run [`sklearn.cluster.AgglomerativeClustering`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) of scikit-learn. We set the `distance_threshold=0` and `n_clusters=None` to calculate and return the full hierarchy of clusters. We also use *Single Linkage* for the linkage method. You can check out the [docs](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) to see the purpose and effects of different input parameters.

In [ ]:
agnes_demo = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='single').fit(X_demo)

Since the clustering is no longer partitional, plotting the results becomes a bit more challenging. But again, scikit-learn provides useful tools to plot dendrograms. A dendrogram is a hierarchical tree-like diagram commonly used to visualize the results of hierarchical clustering algorithms, such as AGNES. It illustrates the arrangement of clusters and their relationships at different levels of similarity or dissimilarity. Dendrograms are particularly useful for understanding the hierarchical structure of the data and identifying the optimal number of clusters.

In a dendrogram, the data points or clusters are represented as leaves or individual branches at the bottom of the diagram. The merging of clusters is depicted through the formation of branches that join together at higher levels of the dendrogram. The height or length of each branch in the dendrogram corresponds to the dissimilarity or distance between the merged clusters. Here's a brief explanation of how to interpret a dendrogram:

* The vertical axis represents the dissimilarity or distance measure used in the clustering algorithm. The scale varies depending on the specific distance metric employed.
* Each branch in the dendrogram represents the merging of clusters at a particular level. The height of the branch indicates the dissimilarity between the clusters being merged. Higher branches indicate larger dissimilarities.
* The order of merging clusters is shown by tracing the branches backward from the leaves towards the root of the dendrogram.
* The horizontal lines in the dendrogram represent the cutoff points that determine the number of clusters. By selecting a horizontal line and cutting the dendrogram at that level, clusters can be obtained.

By visually examining the dendrogram, one can identify different levels of similarity and determine the appropriate number of clusters based on the desired level of granularity. The dendrogram provides an intuitive representation of the clustering process and enables insights into the hierarchical relationships among clusters. 

The following function is directly adopted from [`scikit-learn` website](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html).

In [ ]:
def plot_dendrogram(model, **kwargs):
    plt.figure()
    
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    
    plt.tight_layout()
    plt.show()

Let's run the method with `agnes_demo` to visualize the dendrogram.

In [ ]:
plot_dendrogram(agnes_demo, truncate_mode='level')

The dendrogram should be intuitive to read. From the scatter plot above, we could see that Points 4 and 5 -- strictly speaking: the two clusters that contain Point 4 and Point 5 respectively -- were closest, and thus get merged first. This is followed by merging the two clusters containing Point 0 and Point 1 respectively. Then, according to the dendrogram, Cluster 4/5 got merged with Cluster 3. This might no longer be so obvious from looking at the scatter plot. But keep in mind that Single Linkage was used, so the distance between Point 3 and Point 5 mattered here.

**Side note:** You can try different linkage methods and see how it affects the results. From the lecture, we already know that not all linkage methods will yield the same clustering.

As mentioned above, a typical usage of `AgglomerativeClustering` is to specify the number of clusters (similar to K-Means). Try different values for `n_clusters`.

In [ ]:
agnes_demo = AgglomerativeClustering(n_clusters=3).fit(X_demo)

...and in this case we can plot the clustering as if it is partitional as we are only looking at the level of the hierarchy that resulted in `n_clusters` clusters.

In [ ]:
def plot_clusters(agnes, data, point_size=50, show_ticks=True, aspect=None):
    plt.figure()
    
    if aspect is not None:
        plt.axes().set_aspect(aspect)
    
    # Plot all the data points a color-code them w.r.t. to their cluster label/id
    plt.scatter(data[:, 0], data[:, 1], c=agnes.labels_, s=point_size, cmap=plt.cm.tab10)
    
    if show_ticks is False:
        plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)      
    
    plt.tight_layout()
    plt.show()

Let's have a look.

In [ ]:
plot_clusters(agnes_demo, X_demo)

---

## Playing with Toy Data

[`sklearn.datasets`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets) provides a series of methods to randomly generate sample data. 

Try different methods and see how the results will change.

In [ ]:
X, y = make_blobs(n_samples=100, centers=5, n_features=2, cluster_std=0.85, random_state=11)
#X, y = make_moons(n_samples=250, noise=0.105, random_state=0)
#X, y = make_circles(n_samples=500, noise=0.06, factor=0.5, random_state=0)

plt.figure()
plt.scatter(X[:,0], X[:,1])
plt.show()

### Complete Dendrogram

In the code cell below, we first compute again the complete hierarchy and plot the dendrogram. Even with only 100 data points, the resulting dendrogram showing the full hierarchy already becomes quite cluttered and unreadable.

In [ ]:
agnes_toy = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(X)

plot_dendrogram(agnes_toy, truncate_mode='level')

### Cluster Evaluation

If we specify the number of clusters using the input parameter `n_clusters`, we can evaluate the resulting clusterings the same way as we have seen with K-Means and DBSCAN, namely calculating the Silhouette Score and Rand Index.

Feel free to try out different linkage methods; see the documentation linked above. You also use different types of toy data and see their effects on the results.

In [ ]:
silhouette, ari = [], []

for n_clusters in tqdm(range(2, 20+1)):

    # Run K-Means for the current number of clusters k
    agnes = AgglomerativeClustering(n_clusters=n_clusters, linkage='single').fit(X)
    
    # silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters    
    silhouette.append((n_clusters, silhouette_score(X, agnes.labels_)))
    
    ari.append((n_clusters, adjusted_rand_score(y, agnes.labels_)))
    
# Convert to numpy array for convenience
silhouette = np.array(silhouette)    
ari = np.array(ari)

First, let's print the Silhouette Score.

In [ ]:
plt.figure()
plt.xlabel('Number of Clusters', fontsize=16)
plt.ylabel('Silhouette Score', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in silhouette], [s[1] for s in silhouette], marker='o', lw=2)
plt.tight_layout()
plt.show()

...and now for the Rand Index

In [ ]:
plt.figure()
plt.xlabel('Number of Clusters', fontsize=16)
plt.ylabel('Rand Index', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in ari], [s[1] for s in ari], marker='o', lw=2)
plt.tight_layout()
plt.show()

**For `make_blobs()` with the default values:** Unsurprisingly, for `n_clusters=5`, we see the highest Silhouette Score and the highest Rand Index. It is not surprising since our toy data consists of 5 rather well-separated clusters.

To plot our best clustering, we can run Agnes with `n_clusters=5` and run the method `plot_clusters()`; see below.

In [ ]:
agnes_toy_best = AgglomerativeClustering(n_clusters=5).fit(X)

plot_clusters(agnes_toy_best, X)

---

## Location Data

As a last example, we use the location data we have already seen in the DBSCAN notebook.

In [ ]:
df = pd.read_csv('data/datasets/singapore/sg-places.csv', header=None)

print(set(df[2]))

Let's pick all the places of one type. The descriptions in the following will refer to restaurants (in more detail: McDonald's restaurants; see below). But feel free to change the type to anything of your interest.

In [ ]:
# Try different place types
place_type = 'restaurant'
#place_type = 'bus_station'
#place_type = 'subway_station' # MRT+LRT stations
#place_type = 'store'

df_places = df[df[2]==place_type]

print(df_places.shape)

df_places.head()

3k+ restaurants is quite a lot, and maybe even too diverse for an analysis. So let's further filter the dataset to only include McDonald's restaurants. We use regular expressions for that. No worries, if you're not familiar with regular expressions. In a nutshell, they allow for filtering using flexible substring matching.

In [ ]:
p = re.compile(r'mcdonald', flags=re.IGNORECASE)
df_places = df_places[[bool(p.search(x)) for x in df_places[0]]]

### Visualization

In [ ]:
num_restaurants, num_attributes = df_places.shape

print('Number of places: {}'.format(num_restaurants))

We need to convert the column containing latitude and longitude to a matrix, i.e., a 2d numpy array for further processing. Note that the resulting order is \[longitude, latitude\], since longitude represents the x variable and latitude the y variable. This doesn't matter for the clustering but it ensures that the plots look alright and are not rotated by 90 degrees.

In [ ]:
X_places = df_places[[4, 3]].to_numpy()

print(X_places[0])

The code cell below provides a neat way that the proportions of the plotted points look nicer. Otherwise, the induced shape of Singapore will be squashed. Since Singapore is so close to the equator, this correction is not really needed, though.

In [ ]:
aspect = 1/np.cos(np.radians(1.0))
print(aspect)

Let's plot all the places (e.g., the 141 McDonald's restaurant). You should be able to recognize the outline of Singapore. Of course, if you pick place types that are much less common and/or can only be found in certain areas, you won't be able to "see" the outline of Singapore.

In [ ]:
plt.figure()
plt.axes().set_aspect(aspect)
plt.scatter(X_places[:,0], X_places[:,1], s=25)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
plt.tight_layout()
plt.show()

First, we can have a look and the complete dendrogram show the complete hierarchy.

In [ ]:
agnes_places = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(X_places)

plot_dendrogram(agnes_places, truncate_mode='level')

...but often, we specify the number of clusters; the code cell below uses 10 clusters, but feel free to change this to see how the results change.

In [ ]:
agnes_places = AgglomerativeClustering(n_clusters=10).fit(X_places)

plot_clusters(agnes_places, X_places, show_ticks=False, aspect=aspect)

Since this data is not labeled, we can basically only check the silhouette scores for different numbers of clusters.

In [ ]:
silhouette= []
for n_clusters in tqdm(range(2, 40+1)):

    # Run K-Means for the current number of clusters k
    agnes = AgglomerativeClustering(n_clusters=n_clusters).fit(X_places)
    
    # silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters    
    silhouette.append((n_clusters, silhouette_score(X_places, agnes.labels_)))
    
# Convert to numpy array for convenience
silhouette = np.array(silhouette)    

As usual, we can plot the Silhouette scores for different numbers of clusters to inspect the results.

In [ ]:
plt.figure()
plt.xlabel('Number of Clusters', fontsize=16)
plt.ylabel('Silhouette Score', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in silhouette], [s[1] for s in silhouette], marker='o', lw=2)
plt.tight_layout()
plt.show()

For this dataset (assuming the 141 McDonald's restaurants), the results are less straightforward -- that is, there is no (very) clear best choice for the number of clusters. Again, this is not that surprising seeing that the data points are much more scattered comparedto our toy data above. However, strictly speaking there is one maximum Silhouette Score, and we can check which choice of `n_clusters` resulted in that score.

In [ ]:
best_run = np.argmax(silhouette[:,1])
best_n_clusters = int(silhouette[best_run][0])

print('Number of clusters with the highest Silhouette Score: {}'.format(best_n_clusters))

If we want to, we can again compute AGNES using this best parameter value and plot all clusters.

In [ ]:
agnes_places_best = AgglomerativeClustering(n_clusters=best_n_clusters).fit(X_places)

plot_clusters(agnes_places_best, X_places, show_ticks=False, aspect=aspect)

---

## Summary

AGNES (Agglomerative Nesting) clustering algorithm is an agglomerative hierarchical clustering approach that starts with each data point as a separate cluster and iteratively merges the closest clusters until a termination condition is met. One of the advantages of AGNES is its ability to handle non-globular and irregularly shaped clusters, making it suitable for a wide range of data distributions. It produces a dendrogram, which provides a hierarchical structure of the clusters and allows flexibility in selecting the desired number of clusters by cutting the dendrogram at an appropriate level.

However, AGNES has a few limitations. First, its computational complexity can be high, especially for large datasets, as it requires calculating the proximity matrix at each iteration. This can result in increased time and memory requirements. Second, AGNES is sensitive to the choice of distance metric. The selection of an appropriate distance measure is crucial as it can significantly affect the clustering results. Additionally, AGNES suffers from the "chaining" effect, where once two clusters are merged, they cannot be separated again. This can lead to suboptimal cluster assignments, especially if the initial merging decisions were based on noisy or inaccurate data.

In summary, AGNES is a hierarchical clustering algorithm that offers flexibility in determining the number of clusters through the use of a dendrogram. It can handle complex data distributions and provides insights into the hierarchical relationships among clusters. However, it has computational complexity concerns, is sensitive to the choice of distance metric, and may suffer from the chaining effect. These considerations should be taken into account when applying AGNES in practice, especially for large datasets or when dealing with noisy or ambiguous data.